<a href="https://colab.research.google.com/github/penguinman-99/Python_AI/blob/main/TitanicPredication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#matplotlib.pyplot: 차트를 그려준다.

import matplotlib.pyplot as plt
import numpy as np

x=[1,2,2.5,3,4]
y=[1,4,7,9,15]
plt.plot(x,y,'ro')
plt.axis([0,6,0,20])

In [ ]:
pip install -q sklearn

In [6]:
import os
import sys
#numpy: 수치처리  six.moves:
#pandas:데이터 처리 Ipython.display:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf


In [10]:
# 데이터셋 불러오기.
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
#불러온 데이터 항목에서 생존 항목을 삭제
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')


In [ ]:
dftrain.head()

In [ ]:
#행, 열을 보여준다.
dftrain.shape

#dftrain.age.hist(bins=20)
#성별에 따른 생존률을 보여준다
#dftrain.sex.value_counts().plot(kind='barh')

In [ ]:
#카테고리를 숫자화한다
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))


def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function(): #내부함수.
  #dataset을 이용 label, data가 있는 객체 생성
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)#셔플을 이용한 난수 생성
    ds = ds.batch(batch_size).repeat(num_epochs)
    #한번에 학습할 작은 데이터 사이즈를 32개로 하고, 에포크 수에 대해 프로세스 반복
    return ds
  return input_function

train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)
#앞에 만든 feature column을 전달해 LinearClassifier 모델을 만듬.

linear_est=tf.estimator.LinearClassifier(feature_columns=feature_columns)

# 모델 트레이닝 하기
linear_est.train(train_input_fn)  # 트레이닝
result = linear_est.evaluate(eval_input_fn)  # 테스팅 데이터를 테스트하여 모델 지표 / 통계 가져 오기

clear_output()  # 콘솔 출력 지우기
print(result['accuracy'])  # 모델에 대한 정확도

predicts=list(linear_est.predict(eval_input_fn))


In [36]:

#살아날 확률.
print(dfeval.loc[0])
print(predicts[0]['probabilities'][1])
print(y_eval.loc[3])
print(dfeval.loc[3])
print(predicts[3]['probabilities'][1])

sex                          male
age                            35
n_siblings_spouses              0
parch                           0
fare                         8.05
class                       Third
deck                      unknown
embark_town           Southampton
alone                           y
Name: 0, dtype: object
0.07408214
1
sex                        female
age                            55
n_siblings_spouses              0
parch                           0
fare                           16
class                      Second
deck                      unknown
embark_town           Southampton
alone                           y
Name: 3, dtype: object
0.60438937
